## Librairies

In [ ]:
import numpy as np
import pandas as pd


import os
from glob import glob

import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

## removes tensorflow warning
import tensorflow as tf 

import IPython.display as display



Let's first define some variables, counting images from TFRecords might be fastidious, so i directly computed sizes


In [ ]:
print(tf.__version__)
print("Num GPUs available : ", len(tf.config.list_physical_devices('GPU')))

BATCH = 32
NB_CLASS = 104
EPOCH = 18
TRAIN_IMAGE_NUMBER = 12753
VAL_IMAGE_NUMBER = 3712
TEST_IMAGE_NUMBER = 7382

## Loading files

In [ ]:
local_path = os.path.join('/kaggle','input','tpu-getting-started','tfrecords-jpeg-224x224')


train_filenames = [ fileu for fileu in glob(os.path.join(local_path,'train/*'))]
val_filenames = [ fileu for fileu in glob(os.path.join(local_path,'val/*'))]
test_filenames = [ fileu for fileu in glob(os.path.join(local_path,'test/*'))]


print(f'Total Training TFRECORD FILES : {len(train_filenames)}')
print(f'Total Validation TFRECORD FILES : {len(val_filenames)}')
print(f'Total Testing TFRECORD FILES : {len(test_filenames)}')

### SETUP for TPU utilisation


1. `num_parallel_reads=AUTO` instructs the API to read from multiple files if available. It figures out how many automatically.
2. `experimental_deterministic = False` disables data order enforcement. We will be shuffling the data anyway so order is not important. With this setting the API can use any TFRecord as soon as it is streamed in.
3. However for predictions we should keep the order so this variable must be set to `True`

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
ignore_order = tf.data.Options()

ignore_order.experimental_deterministic = False


In [ ]:
def decode_image(image):
    
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.cast(image,tf.float32)
    image = tf.reshape(image,[224,224,3])
    
    return image


def read_training_tfrecord(example_proto):
    image_feature_description = {
        'class' : tf.io.FixedLenFeature([] , tf.int64),
        'image' : tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example_proto, image_feature_description)
    
    image = decode_image(example['image'])
    
    label = tf.cast(example['class'], tf.int32)
    
    label = tf.one_hot(label, NB_CLASS)
        
        
    return image, label


def read_testing_tfrecord(example_proto):
    
    image_feature_description = {
        'id' : tf.io.FixedLenFeature([], tf.string),
        'image' : tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example_proto, image_feature_description)
    
    image = decode_image(example['image'])
    
    ids = example['id']
    
    return image, ids
    
    
 ## We do not want to the data to be shuffled for testing, so we separate process   
def load_dataset(filenames,style):
    
    
    dataset = tf.data.TFRecordDataset(filenames,num_parallel_reads=AUTO)
    
    
    if style =='training' or style=='validation' : 
        
        dataset = dataset.with_options(ignore_order)
    
        dataset = dataset.map(read_training_tfrecord)
            
        dataset = dataset.cache().shuffle(1000).prefetch(buffer_size=32)

        dataset = dataset.batch(BATCH)
        
    else :
        
        ignore_order.experimental_deterministic = True
        
        dataset = dataset.with_options(ignore_order)
        
        dataset = dataset.map(read_testing_tfrecord)
        
        dataset = dataset.prefetch(buffer_size=32).batch(BATCH)
        
    
    return dataset

    

In [ ]:
train_dataset = load_dataset(train_filenames,style = 'training')

val_dataset = load_dataset(val_filenames, style = 'validation')


test_dataset = load_dataset(test_filenames, style = 'testing')

## Simple Visualisation

In [ ]:
def display_batch(image_batch,label_batch):
    plt.figure(figsize=(10,10))
    
    for k in range(16):
        
        ax = plt.subplot(4,4,k+1)
        
        plt.imshow(image_batch[k] /255.)
        
        
        #plt.title('Classes : '+ str(id_batch[k]))
        #because one_hot_encoding applied
        plt.axis('off')

image_batch, label_batch = next(iter(train_dataset))     
display_batch(image_batch, label_batch)
        

### Building model


For the model, let's use Xception pre-trained model on imageNet, Inception module is really powerful.  
Concerning top layers, no real research/comparison were performed. So one way to improve the model would be to try different architecture. 

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import schedules

import tensorflow_addons as tfa



## Possible imporvements

# - Using sparseCorssEntropy instead of hot encoding
# - training also the base model
# - Solid LearningRateScheduler

In [ ]:

def build_model():
    
    base_model = tf.keras.applications.Xception(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=(224,224,3),
    )
    
    base_model.trainable = False
        
    data_aug = Sequential([
        
        layers.Rescaling(1./127.5, offset=-1),
        
        layers.RandomFlip("horizontal_and_vertical"),
        
        layers.RandomRotation(0.3)])
    
    model = Sequential([
        
        data_aug,
              
        base_model,
        
        layers.GlobalAveragePooling2D(),
    
        layers.Dense(1024, activation = 'relu'),
    
        layers.Dropout(0.2),
        
        layers.Dense(256, activation = 'relu'),
        
        layers.Dropout(0.2),
    
        layers.Dense(NB_CLASS,activation = 'softmax')
    ])
    

    
    return model

In [ ]:
class PrintLR(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print('\nLearning rate for epoch {} is {}'.format(epoch + 1,model.optimizer.lr.numpy()))
##not really usefull

1. Early Stopping isn't really useful here, because i didn't trained on many epoch
2. The ExponnetialDecay is also not really useful, however one must note that even if Adam optimizer modify itself learning rate, adding a scheduler fixes the upper limit of learning rate modified 
3. About the metric, again it is not useful to work with F1_score directly, however it gives a glimpse of your model capacity ( because this is the exact metric used by kaggle on this competition

In [ ]:
def training():

    init_lr = 0.001

    
    lr_scheduler = schedules.ExponentialDecay(
        init_lr, decay_steps=int(np.ceil(TRAIN_IMAGE_NUMBER/BATCH)), decay_rate=0.94, staircase=True)

    early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        patience=3, restore_best_weights=True)

    checkpoint_path = './training_1/cp.ckpt'

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path,save_weights_only=True, save_best_only=True, verbose = 1
    )

    with tf.device('/device:GPU:0'):
        model = build_model()

        model.compile(optimizer=keras.optimizers.Adam(learning_rate = lr_scheduler),
                      loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                      metrics=tfa.metrics.F1Score(num_classes=NB_CLASS, average = 'macro', threshold=0.5)
        )
   
        
    history = model.fit(
        train_dataset,
        epochs=EPOCH,
        validation_data=val_dataset,
        callbacks=[checkpoint_cb, early_stopping_cb],
)
    
    return model

## Model Predictions

In [ ]:
def test(model):
    testing_image = test_dataset.map(lambda image, ids : image)

    testing_ids = test_dataset.map(lambda image, ids: ids).unbatch()


    test_ids = next(iter(testing_ids.batch(TEST_IMAGE_NUMBER))).numpy().astype('U')


    predictions_raw = model.predict(testing_image, batch_size = BATCH)

    pred = np.argmax(predictions_raw, axis=-1)


    predictions_raw = model.predict(testing_image, batch_size = BATCH)

    pred = np.argmax(predictions_raw, axis=-1)
    
    
    submission_df = pd.DataFrame(data ={'id': test_ids, 'label' : pred} ).set_index('id')


    submission_df.to_csv('submission.csv')

    return


In [ ]:
model = training()

In [ ]:
test(model)